# Example usage

To use `labtools` in a project:

## Imports

In [10]:
from labtools import shuffle

## Shuffle a sequence

Create shuffles of the alphabet:

In [2]:
shuffles_list, names_list = shuffle.windowed_shuffle("ABCDEFGHIJKLMNOPQRSTUVWYXZ")
shuffles_list

['dabceFGHIJKLMNOPQRSTUVWYXZ',
 'AedbfcGHIJKLMNOPQRSTUVWYXZ',
 'ABgdefcHIJKLMNOPQRSTUVWYXZ',
 'ABCgdfehIJKLMNOPQRSTUVWYXZ',
 'ABCDfgiheJKLMNOPQRSTUVWYXZ',
 'ABCDEfgjhiKLMNOPQRSTUVWYXZ',
 'ABCDEFjgkhiLMNOPQRSTUVWYXZ',
 'ABCDEFGjihlkMNOPQRSTUVWYXZ',
 'ABCDEFGHikljmNOPQRSTUVWYXZ',
 'ABCDEFGHIlmknjOPQRSTUVWYXZ',
 'ABCDEFGHIJlmnkoPQRSTUVWYXZ',
 'ABCDEFGHIJKlonpmQRSTUVWYXZ',
 'ABCDEFGHIJKLpnmoqRSTUVWYXZ',
 'ABCDEFGHIJKLMqrponSTUVWYXZ',
 'ABCDEFGHIJKLMNorsqpTUVWYXZ',
 'ABCDEFGHIJKLMNOtpqsrUVWYXZ',
 'ABCDEFGHIJKLMNOPuqstrVWYXZ',
 'ABCDEFGHIJKLMNOPQusvrtWYXZ',
 'ABCDEFGHIJKLMNOPQRtsuvwYXZ',
 'ABCDEFGHIJKLMNOPQRSutvywXZ',
 'ABCDEFGHIJKLMNOPQRSTwyxvuZ',
 'ABCDEFGHIJKLMNOPQRSTUyvwzx']

In [11]:
from labtools.adtools import seqlib

In [12]:
from labtools.adtools.finder import pull_AD

In [7]:
pull_AD("ACTTTTATVGCTAGCATGGCTGGTAGATCTTGGTTGATTGATTCTAATAGAATTGCTACTAAGATTATGTCTGCTTCTGCTTCTTCTGATCCAAGACAAGTTGTTTGGAAATCTAATCCATCTAGACATTGTCCAGCTGATCGATGCTAGTAGAGAGAGA")

('ATGGCTGGTAGATCTTGGTTGATTGATTCTAATAGAATTGCTACTAAGATTATGTCTGCTTCTGCTTCTTCTGATCCAAGACAAGTTGTTTGGAAATCTAATCCATCTAGACATTGTCCA',
 None)

In [21]:
from labtools.adtools.pair_counter import seq_counter
seq_counter("../exampledata/mini.fastq")

GGTTCTTCTAAATTGAGATGTGATAATAATGCTGCTGCTCATGTTAAATTGGATTCATTTCCAGCTGGTGTTAGATTTGATACATCTGATGAAGAATTGTTGGAACATTTGGCTGCTAAA    1
GAAGAATTGTTTTTACATTTGTCTGCTAAGATTGGTAGATCTTCTAGGAAACCACATCCATTCTTGGATGAATTTATTCATACTTTGGTTGAAGAAGATGGTATTTGTAGAACTCATCCA    3
dtype: int64